In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')

In [3]:
!pip install transformers

     |████████████████████████████████| 2.2MB 8.0MB/s 
     |████████████████████████████████| 870kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 53.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=a8b30bd69a82d6c0e4bdfe84c5e1bb71da69a5c80c73d39ecf7341814c7cb189
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [5]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 9.7MB/s 


In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import *
import numpy as np
import argparse
from tqdm import tqdm 
import random
import transformers
import pandas as pd
from collections import Counter
from transformers import get_linear_schedule_with_warmup
import sys
sys.argv=['']
del sys

In [7]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [8]:
len(X)

33112

In [9]:
X[3:8]

array(['Mình gọi 2 cái lẩu + lẩu nấm\n+ lẩu hải sản\nDĩa nấm nhiều loại nhưng nấm đông cô hơi có mùi hăng .\nNước dùng của 2 lẩu y hệt nhau , lẩu hải sản không gì đặt biệt ... Nói chung là tạm thôi',
       '# Tetyeuthuong Chúc toàn thể cán bộ, anh chị em # BIDV nói riêng và tất cả mọi người trên mọi miền tổ quốc nói chung có một cái Tết sum vầy, đầm ấm bên gia đình và người thân, gặt hái nhiều thành công trong năm mới',
       'Hơi cay đúng vị lẩu thái , bún ngon và đầy đặn , đặc biệt thích con bề bề : *',
       'Đi ăn vào một buổi trưa nắng nhẹ . Thấy lạ nên mình gọi Mì Quảng Ếch . Lạ lạ nhưng vị thì hông có gì đặc sắc lắm .',
       'Lúc bạn mình mua nước về , nhìn những chai nước bé bé xinh xinh cực kết luôn\nThế nhưng hỡi ôi khi uống zô mới biết , dở không chịu nổi từ sake , bí đao , đậu đen ... chỉ có sữa bắp với nha đam là tạm ổn .\nVới giá 20k mình thấy như thế là ko đáng\nQuá dở'],
      dtype=object)

In [15]:
X[106:112]

array(['Quán ăn được , phục vụ chậm , kêu nước 2 lần mà hẹn quài không bưng ra . Tính tiền thì hỏi 2 em không uống nước à ( cười nhết mém ) , 2 e không dùng khăn luôn à ( cười nhết mém ) . Nói chung quán bán thì ăn ngon , nhưng xem lại cách phục vụ , chúng em không có nhu cầu dùng khăn thì đừng tỏ thái độ như bọn em không có tiền .',
       'Xiên nướng tạm đc , k dở cũng k ngon , rau cho 1 dĩa mà sâu k , chả ăn đc , thái độ nv tệ , chỉ có chú gửi xe vs chị bé đứnng nướng đồ là thấy đc . Tóm lại là tệ !',
       'Cà phê ở đay uống ngon , cà phê máy phong cách Ý , có đủ loại cho người ghiền cà phê như mình . Đi làm về là tạt ngang làm một cốc Capuchino uống cho đỡ căng thẳng , tối về uống khỏi ngủ luôn\nRảnh rảnh cuối tuần mình với bạm cũng hay ghé đây uống cà phê trà đào . Ngon , mát , đẹp , nói chung là thích .\nMặt bằng đẹp - Cà phê ngon - không có gì khác biệt : ) ) )',
       'Ghé quán cũng 3 lần rồi mà ko lần nào được ăn món " đặc biệt nhất " của quán , quán nhân viên phục vụ khá đ

In [16]:
import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [17]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [18]:
X = transform(X)
X = remove_duplicate(X)

In [19]:
X[106:112]

['quán ăn được , phục vụ chậm , kêu nước 2 lần mà hẹn quài không bưng ra . tính tiền thì hỏi 2 em không uống nước à ( cười nhết mém ) , 2 e không dùng khăn luôn à ( cười nhết mém ) . nói chung quán bán thì ăn ngon , nhưng xem lại cách phục vụ , chúng em không có nhu cầu dùng khăn thì đừng tỏ thái độ như bọn em không có tiền .',
 'xiên nướng tạm đc , k dở cũng k ngon , rau cho 1 dĩa mà sâu k , chả ăn đc , thái độ nv tệ , chỉ có chú gửi xe vs chị bé đứng nướng đồ là thấy đc . tóm lại là tệ !',
 'cà phê ở đay uống ngon , cà phê máy phong cách ý , có đủ loại cho người ghiền cà phê như mình . đi làm về là tạt ngang làm một cốc capuchino uống cho đỡ căng thẳng , tối về uống khỏi ngủ luôn\nrảnh rảnh cuối tuần mình với bạm cũng hay ghé đây uống cà phê trà đào . ngon , mát , đẹp , nói chung là thích .\nmặt bằng đẹp - cà phê ngon - không có gì khác biệt : ) ) )',
 'ghé quán cũng 3 lần rồi mà ko lần nào được ăn món " đặc biệt nhất " của quán , quán nhân viên phục vụ khá đc nhưng đồ ăn thì phải có

In [20]:
!pip install emoji_translate

     |████████████████████████████████| 430kB 7.4MB/s 
     |████████████████████████████████| 133kB 14.5MB/s 


In [21]:
from emoji_translate.emoji_translate import Translator

emo = Translator(exact_match_only=False, randomize=True)

In [23]:
print(emo.remove_positive_emojis('#tetyeuthuong năm 🐵 qua đi nằm 🐔 lại đến chúc tất cả ace bidv nói riêng và cả nước nói chung 1 năm mới an khang thịnh vượng, sức khỏe dồi dào'))

#tetyeuthuong năm 🐵 qua đi nằm 🐔 lại đến chúc tất cả ace bidv nói riêng và cả nước nói chung 1 năm mới an khang thịnh vượng, sức khỏe dồi dào


In [24]:
emo.remove_emoji_by_polarity('#tetyeuthuong năm 🐵 qua đi nằm 🐔 lại đến chúc tất cả ace bidv nói riêng và cả nước nói chung 1 năm mới an khang thịnh vượng, sức khỏe dồi dào')

'#tetyeuthuong năm qua đi nằm lại đến chúc tất cả ace bidv nói riêng và cả nước nói chung 1 năm mới an khang thịnh vượng, sức khỏe dồi dào'

In [25]:
emo.remove_emoji_by_polarity('quán ruột sau khi ăn ở bên 533/59 ốc luộc huỳnh văn bánh bị đau bụng , h thì lên giá rùi nhưng vẫn thích ăn càng ghẹ 30k nhâm nhi ngon quá chừng . bữa nay mình đi ăn thì k có nhiều loại nữa , ốc hương ốc mỡ cũng k bán , bạch tuộc ở đây quá mắc , , 50k , ăn ngon nhưng quá ít . dù gì thì mình vẫn quay lại : ) ) ) )')

'quán ruột sau khi ăn ở bên 533/59 ốc luộc huỳnh văn bánh bị đau bụng , h thì lên giá rùi nhưng vẫn thích ăn càng ghẹ 30k nhâm nhi ngon quá chừng . bữa nay mình đi ăn thì k có nhiều loại nữa , ốc hương ốc mỡ cũng k bán , bạch tuộc ở đây quá mắc , , 50k , ăn ngon nhưng quá ít . dù gì thì mình vẫn quay lại : ) ) ) )'

In [26]:
train_text = []
for text in X:
  train_text.append(emo.remove_emoji_by_polarity(text))
train_text

['quán nhỏ , tối tối , dễ tìm nhưng trông rất luộm thuộm , lộ hẳn gian bếp gia đình trông bẩn bẩn . tầng 2 điều hòa mát rượi nhưng nhìn đâu cũng thấy bụi bẩn : sách trên kệ trông như ngàn năm không ai đụng , thảm nhăn nhúm đầy vụn rác , lúc mới vào còn có mùi khai như mùi mèo , đến miệng cốc nước lọc cũng rửa không sạch = = phục vụ siêu lâu , để cái thẻ thanh toán như vứt lên bàn . đồ uống tạm đc .',
 'nghe bảo trà nổi tiếng nhưng uống nhật thua cả bobabop hay tiên hưởng nhiều mà giá cả chả rẻ lúc nào cũng đông',
 '100k được 40 cái tính ra cũng không quá mắc . chị bán hàng cũng nhiệt tình . nhưng tiệm hơi khó tìm vì bị thụt vào trong . bánh hơi nhỏ so với tưởng tượng , bù lại được cái xinh thôi rồi',
 'mình gọi 2 cái lẩu + lẩu nấm\n+ lẩu hải sản\ndĩa nấm nhiều loại nhưng nấm đông cô hơi có mùi hăng .\nnước dùng của 2 lẩu y hệt nhau , lẩu hải sản không gì đặt biệt ... nói chung là tạm thôi',
 '# tetyeuthuong chúc toàn thể cán bộ, anh chị em # bidv nói riêng và tất cả mọi người trên mọi 

In [27]:
len(train_text)

33112

In [ ]:
import pickle
import re

with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convert_emojis_to_word(text):
#     for emot in Emoji_Dict:
#         text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
#     return text

# text = "I won 🥇 in 🏏"
# convert_emojis_to_word(text)

In [ ]:
def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
      if emot in text:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

text = "I won 🥇 in 🏏"
convert_emojis_to_word(text)

'I won 1st_place_medal in cricket'

In [28]:
import re
import pickle  

with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in Emoticon_Dict) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Good Morning :-)")

'Good Morning '

In [31]:
remove_emoticons('cực thích bánh tiramisu và su kem ở đây , vừa ăn không ngọt - nhưng kh thích bánh bao kim sa ở đây : (')

'cực thích bánh tiramisu và su kem ở đây , vừa ăn không ngọt - nhưng kh thích bánh bao kim sa ở đây : ('

In [30]:
final_text = []
for text in train_text:
  final_text.append(remove_emoticons(text))
final_text

['quán nhỏ , tối tối , dễ tìm nhưng trông rất luộm thuộm , lộ hẳn gian bếp gia đình trông bẩn bẩn . tầng 2 điều hòa mát rượi nhưng nhìn đâu cũng thấy bụi bẩn : sách trên kệ trông như ngàn năm không ai đụng , thảm nhăn nhúm đầy vụn rác , lúc mới vào còn có mùi khai như mùi mèo , đến miệng cốc nước lọc cũng rửa không sạch = = phục vụ siêu lâu , để cái thẻ thanh toán như vứt lên bàn . đồ uống tạm đc .',
 'nghe bảo trà nổi tiếng nhưng uống nhật thua cả bobabop hay tiên hưởng nhiều mà giá cả chả rẻ lúc nào cũng đông',
 '100k được 40 cái tính ra cũng không quá mắc . chị bán hàng cũng nhiệt tình . nhưng tiệm hơi khó tìm vì bị thụt vào trong . bánh hơi nhỏ so với tưởng tượng , bù lại được cái xinh thôi rồi',
 'mình gọi 2 cái lẩu + lẩu nấm\n+ lẩu hải sản\ndĩa nấm nhiều loại nhưng nấm đông cô hơi có mùi hăng .\nnước dùng của 2 lẩu y hệt nhau , lẩu hải sản không gì đặt biệt ... nói chung là tạm thôi',
 '# tetyeuthuong chúc toàn thể cán bộ, anh chị em # bidv nói riêng và tất cả mọi người trên mọi 

In [32]:
len(final_text)

33112

In [33]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--maxlen', type=int, default=120)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--lr', type=float, default=2e-5)
args = parser.parse_args()

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

MAX_LEN = args.maxlen

In [34]:
MAX_LEN

120

In [35]:
def get_dataloader(sentences, labels, tokenizer, batch_size):
    input_ids = []
    filtered_labels = []
    for i, sent in tqdm(enumerate(sentences)):
        encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
        if len(encoded_sent) <= 512:
            input_ids.append(encoded_sent)
            filtered_labels.append(labels[i])
    print(Counter(filtered_labels))
    inputs = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                                    value=0, truncating="post", padding="post")                                                     
    masks = []

    for sent in inputs:
        att_mask = [int(token_id > 0) for token_id in sent]
        masks.append(att_mask)

    inputs = torch.tensor(inputs)
    labels = torch.tensor(filtered_labels)
    masks = torch.tensor(masks)

    data = TensorDataset(inputs, masks, labels)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

In [36]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [37]:
def get_baseline_acc(model, validation_dataloader, device):
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    eval_accuracy = eval_accuracy/nb_eval_steps
    return eval_accuracy

In [38]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [40]:
X[1]

'nghe bảo trà nổi tiếng nhưng uống nhật thua cả bobabop hay tiên hưởng nhiều mà giá cả chả rẻ lúc nào cũng đông'

In [41]:
final_text[1]

'nghe bảo trà nổi tiếng nhưng uống nhật thua cả bobabop hay tiên hưởng nhiều mà giá cả chả rẻ lúc nào cũng đông'

In [42]:
X_train = final_text[:-10000]
Y_train = Y[:-10000]
X_test = final_text[-10000:]
Y_test = Y[-10000:]
print(Counter(Y_train), Counter(Y_test))
train_dataloader = get_dataloader(X_train, Y_train, tokenizer, args.batch_size)
test_dataloader = get_dataloader(X_test, Y_test, tokenizer, args.batch_size)

optimizer = AdamW(model.parameters(),lr = args.lr)
total_steps = len(train_dataloader) * args.epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = int(0.1*total_steps),
                                            #warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
                                            #t_total = total_steps)

101it [00:00, 1005.30it/s]

Counter({2: 11721, 0: 11033, 1: 358}) Counter({2: 5101, 0: 4742, 1: 157})


9617it [00:09, 979.24it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
23112it [00:23, 974.52it/s]


Counter({2: 11721, 0: 11032, 1: 358})


10000it [00:10, 968.50it/s]


Counter({2: 5101, 0: 4741, 1: 157})


In [43]:
print(len(X_train))
print(len(X_test))

23112
10000


In [44]:
for epoch_i in range(args.epochs):
    total_loss = 0
    model.train()
        
    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader)):
        model.train()
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
                
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        loss.backward()

        total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    test_accuracy = get_baseline_acc(model, test_dataloader, device)
    print(" Test acc {}".format( test_accuracy))

print("")
print("Training complete!")

2889it [10:45,  4.47it/s]



  Average training loss: 0.54


0it [00:00, ?it/s]

 Test acc 0.8334714285714286


2889it [10:46,  4.47it/s]


...


  Average training loss: 0.04


0it [00:00, ?it/s]

 Test acc 0.8594857142857143


2889it [10:32,  4.57it/s]



  Average training loss: 0.03
 Test acc 0.8642

Training complete!
